In [2]:
import pickle
import sqlparse

from django.test.client import RequestFactory
from django.db import connection, reset_queries

from cast.cache import PagedPostData

# Replace ORM queries with raw SQL

Why? Because it's difficult to cache querysets. They tend to contain
generators, nested classes, the gorilla holding the banana and the whole jungle.

A database fetchall result on the other hand is just a list of dicts.
This should be easy to cache.

In [3]:
def show_queries(queries):
    for query in queries:
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        print("------------------------------------------------")
        print(formatted_sql)
        

def blocker(*args):
    raise Exception("No database access allowed here.")

    
# blog_slug = "ephes_blog"
blog_slug = "das_claas_und_nora_blog"
blog = Blog.objects.get(slug=blog_slug)
request_factory = RequestFactory()

In [4]:
# %%time
# reset_queries()
# post_queryset = blog.unfiltered_published_posts
# post_data = PagedPostData.data_for_blog_index(
#     request=request_factory.get("/"),
#     blog=blog,
# )
# print(len(connection.queries))

In [5]:
#show_queries(connection.queries)

# Just get cachable data

In [6]:
%%time
request = request_factory.get(blog.get_url())
request.htmx = False
reset_queries()
cachable_data = PagedPostData.data_for_blog_index_cachable(request=request, blog=blog)
print(len(connection.queries))

31
CPU times: user 121 ms, sys: 20.2 ms, total: 141 ms
Wall time: 169 ms


In [6]:
%%time
cache_path = "paged_post_data.pkl"
with open(cache_path, "wb") as f:
    pickle.dump(cachable_data, f)

with open(cache_path, "rb") as f:
    loaded = pickle.load(f)

CPU times: user 3.53 ms, sys: 3.37 ms, total: 6.9 ms
Wall time: 4.11 ms


In [7]:
%%time
paged_post_data = PagedPostData.create_from_cachable_data(data=loaded)

CPU times: user 43.7 ms, sys: 12.6 ms, total: 56.3 ms
Wall time: 60.5 ms


In [8]:
response = blog.serve(request, post_data=paged_post_data)

In [9]:
paged_post_data.template_base_dir

'bootstrap5'

In [10]:
paged_post_data

# With cached post data vs no cache

## without

In [15]:
%%time
request = request_factory.get(blog.get_url() + "?foo=bar")
request.htmx = False
content = blog.serve(request, post_data=paged_post_data).render()

CPU times: user 50.8 ms, sys: 20.1 ms, total: 71 ms
Wall time: 75.3 ms


## with

In [16]:
%%time
request = request_factory.get(blog.get_url())
request.htmx = False
content = blog.serve(request, post_data=paged_post_data).render()

CPU times: user 44.8 ms, sys: 6.4 ms, total: 51.2 ms
Wall time: 52.6 ms


In [8]:
foo = '<a class="text-dark text-decoration-none" href="/blogs/ephes_blog/weeknotes-2024-04-01/">Weeknotes 2024-04-01</a>'

In [13]:
post = Post.objects.first()

In [14]:
post.url

'/blogs/das_claas_und_nora_blog/Zu-ersten-Mal-etwas-Mohrenbrei/'

In [15]:
post.page_url

AttributeError: 'Post' object has no attribute 'page_url'